In [1]:
import re
import pickle

loadpath = './test.csv'
x = []
for line in open(loadpath,"rb"):
    x.append(line)

In [2]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"\'s", " is", string)
    string = re.sub(r"[^A-Za-z0-9(),\.!?]", " ", string)
    #string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"e\.g\.,", " ", string)
    string = re.sub(r"a\.k\.a\.", " ", string)
    string = re.sub(r"i\.e\.,", " ", string)
    string = re.sub(r"i\.e\.", " ", string)
    #string = re.sub(r"\'ve", " \'ve", string)
    #string = re.sub(r"\'", "", string)
    #string = re.sub(r"\'re", " \'re", string)
    #string = re.sub(r"\'d", " \'d", string)
    #string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"br", "", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " ( ", string)
    string = re.sub(r"\)", " ) ", string)
    string = re.sub(r"\?", " ? ", string)
    string = re.sub(r"\.", " . ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"u\.s\.", " us ", string)
    return string.strip().lower()

In [3]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [4]:
lab = []      #测试集标签
sent = []     #测试集句子
vocab = {}    #词典

In [5]:
for i in range(60000):
    line = str(x[i]).rstrip("\\n\''")
    index = [i for i in range(len(line)) if line[i] == ","]
    sent_start_index = index[2]
    temp = clean_str(line[sent_start_index+1:]).split()
    temp = [ j if not is_number(j) else '0' for j in temp]
    if len(temp) > 300:
        lab.append(clean_str(line.split(",")[0]).replace("b ",""))
        temp = temp[:300]
        sent.append(temp)
    elif len(temp) <= 5: # remove too short question
        continue
    else:
        lab.append(clean_str(line.split(",")[0]).replace("b ",""))
        sent.append(temp)
    
    for word in temp:
        if word in vocab:
            vocab[word] += 1
        else:
            vocab[word] = 1

In [6]:
loadpath = './train.csv'

x = []
for line in open(loadpath,"rb"):
    x.append(line)

train_lab = []      #测试集标签
train_sent = []     #测试集句子

for i in range(len(x)):
    line = str(x[i]).rstrip("\\n\''")
    index = [i for i in range(len(line)) if line[i] == ","]
    sent_start_index = index[2]
    temp = clean_str(line[sent_start_index+1:]).split()
    temp = [ j if not is_number(j) else '0' for j in temp]
    if len(temp) > 300:
        train_lab.append(clean_str(line.split(",")[0]).replace("b ",""))
        temp = temp[:300]
        train_sent.append(temp)
    elif len(temp) <= 5: # remove too short question
        continue
    else:
        train_lab.append(clean_str(line.split(",")[0]).replace("b ",""))
        train_sent.append(temp)
    
    for word in temp:
        if word in vocab:
            vocab[word] += 1
        else:
            vocab[word] = 1

In [7]:
ixtoword = {}
# period at the end of the sentence. make first dimension be end token
ixtoword[0] = 'END'
ixtoword[1] = 'UNK'
wordtoix = {}
wordtoix['END'] = 0
wordtoix['UNK'] = 1
ix = 2
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [8]:
def convert_word_to_ix(data):
    result = []
    for sent in data:
        temp = []
        for w in sent:
            if w in wordtoix:
                temp.append(wordtoix[w])
            else:
                temp.append(1)
        temp.append(0)
        result.append(temp)
    return result

train_x = train_sent[:1100000]
train_y = train_lab[:1100000]
val_x = train_sent[1100000:]
val_y = train_lab[1100000:]
test_x = sent
test_y = lab



train_x = convert_word_to_ix(train_x)
val_x = convert_word_to_ix(val_x)
test_x = convert_word_to_ix(test_x)

In [9]:
pickle.dump([train_x, val_x, test_x, train_y, val_y, test_y, wordtoix, ixtoword], open("yahoo4char.p", "wb"))

In [10]:
train_y[0]

'5'